In [1]:
# Imports (requirements)
import pandas as pd
import numpy as np
import math
import random
import seaborn as sns
import matplotlib.pyplot as plt
import optuna
%matplotlib inline

import warnings
warnings.filterwarnings('ignore')
warnings.simplefilter('ignore')

In [2]:
# Initial attractions (A) - 2x2
A_0_p1s1 = 1 # Player 1's Initial attraction to strategy 1
A_0_p1s2 = 0 # Player 1's initial attraction to strategy 2
A_0_p2s1 = -2 # Player 2's Initial attraction to strategy 1
A_0_p2s2 = 0 # Player 2's Initial attraction to strategy 2

# Initial experience (N)
N_0 = 1

# Rounds
n_sims = 50 # epochs (outer loop)
n_periods = 1000 # rounds within epochs (inner loop)

#TO BE SOLVED
real_prob_p1s1 = 0.5
real_prob_p2s1 = 0.5

In [3]:
# Game Structure Definitions (2x2)

# Defining the matrix game in a vector-like structure
p1_payoffs = [1, 0, 0, 2] # p1's (ROW) payoffs in a bi-matrix arrangement
p2_payoffs = [-1, 0, 0, -2] # p2's (COLUMN) payoffs in a bi-matrix arrangement

# Create a matrix for each of the payers' strategy profiles
p1_mat = p1_payoffs[:2], p1_payoffs[2:]
p2_mat = p2_payoffs[:2], p2_payoffs[2:]

# Probability arrays returned by selected strategies (for payoffs computation via matrix multiplication)
s1_array = np.asarray([1, 0])
s2_array = np.asarray([0, 1])

In [ ]:
# Generation of choice probabilities via logistic transformation of the Attraction weights
def probabilities_generation(lam, A_ij, A_i0, A_i1):
    prob_strategy = np.exp(lam * A_ij) / (np.exp(lam * A_i1) + np.exp(lam * A_i0))
    return prob_strategy

# Strategy selection function
def strategy_selection(prob_i0):
    rand_u = random.uniform(0, 1)
    if rand_u <= prob_i0:
        strat_i = s1_array
    else:
        strat_i = s2_array
    return strat_i

# Generation of possible payoff scenarios, given the players' selected strategies
def payoffs_generation(s1_array, s2_array, p1_mat, p2_mat, strat_p1, strat_p2):
    # Payoffs computation (via matrix multiplication)
    payoff_p1_s1 = np.dot(
        np.dot(s1_array, p1_mat), strat_p2
    )  # Payoff player 1 receive for strategy 1, given the choice of player 2
    payoff_p1_s2 = np.dot(
        np.dot(s2_array, p1_mat), strat_p2
    )  # Payoff player 1 receive for strategy 2, given the choice of player 2
    payoff_p2_s1 = np.dot(
        np.dot(strat_p1, p2_mat), s1_array
    )  # Payoff player 2 receive for strategy 1, given the choice of player 1
    payoff_p2_s2 = np.dot(
        np.dot(strat_p1, p2_mat), s2_array
    )  # Payoff player 2 receive for strategy 2, given the choice of player 1
    return payoff_p1_s1, payoff_p1_s2, payoff_p2_s1, payoff_p2_s2

# Computation of weighted payoffs for attractions calculations based on the players' selected strategies
def weighted_payoffs_generation(
    strat_p1, strat_p2, payoff_p1_s1, payoff_p1_s2, payoff_p2_s1, payoff_p2_s2, delta
):  
    # Defining weights for payoffs based on the selected strategy (to be weighted by delta)
    # Validation done via comparison of numpy arrays
    strategy_p1_comp = strat_p1 == s1_array
    if strategy_p1_comp.all():
        p1_strategy = 1
        I_p1s1 = 1
        I_p1s2 = 0
    else:
        p1_strategy = 2
        I_p1s1 = 0
        I_p1s2 = 1
    wp_p1s1 = (delta + (1 - delta) * I_p1s1) * payoff_p1_s1
    wp_p1s2 = (delta + (1 - delta) * I_p1s2) * payoff_p1_s2
    strategy_p2_comp = strat_p2 == s1_array
    if strategy_p2_comp.all():
        p2_strategy = 1
        I_p2s1 = 1
        I_p2s2 = 0
    else:
        p2_strategy = 2
        I_p2s1 = 0
        I_p2s2 = 1
    wp_p2s1 = (delta + (1 - delta) * I_p2s1) * payoff_p2_s1
    wp_p2s2 = (delta + (1 - delta) * I_p2s2) * payoff_p2_s2
    return p1_strategy, p2_strategy, wp_p1s1, wp_p1s2, wp_p2s1, wp_p2s2

# Attraction calculation formula (weighted payoffs (w) calculated in the weighted_payoffs_generation() function)
def ewa_attraction(phi, A, N, N_new, w):
    A_new = (phi * N * A + w) / N_new
    return A_new

# Attraction update rule - create attraction values for current period (t)
def ewa_update(
    rho, A_p1s1, A_p1s2, A_p2s1, A_p2s2, wp_p1s1, wp_p1s2, wp_p2s1, wp_p2s2, N, phi
):
    N_new = rho * N + 1
    # Update for player 1
    A_p1s1 = ewa_attraction(phi, A_p1s1, N, N_new, wp_p1s1)
    A_p1s2 = ewa_attraction(phi, A_p1s2, N, N_new, wp_p1s2)
    # Update for player 2
    A_p2s1 = ewa_attraction(phi, A_p2s1, N, N_new, wp_p2s1)
    A_p2s2 = ewa_attraction(phi, A_p2s2, N, N_new, wp_p2s2)
    
    return A_p1s1, A_p1s2, A_p2s1, A_p2s2, N_new

# Core simulation function
def ewa_simulation(n_sims, n_periods, A_0_p1s1, A_0_p1s2, A_0_p2s1, A_0_p2s2, N_0, params, trial=None):
    out_row, out_all = [], []
    round_nr = 0
    sim_nr = 1
    trigger_times = 0
    prev_loss = 1e9

    for sim in range(0, n_sims):
        round_nr = 0
        # Re-setting variables to initial value for a new simulation run
        A_p1s1 = A_0_p1s1
        A_p1s2 = A_0_p1s2
        A_p2s1 = A_0_p2s1
        A_p2s2 = A_0_p2s2
        N = N_0
        cum_payoff_p1 = 0
        cum_payoff_p2 = 0

        for period in range(0, n_periods):
            # running 'n_periods' epochs
            # Generate choice probabilities based on Attractions
            prob_p1s1 = probabilities_generation(params["lam"], A_p1s1, A_p1s1, A_p1s2)
            prob_p1s2 = probabilities_generation(params["lam"], A_p1s2, A_p1s1, A_p1s2)
            prob_p2s1 = probabilities_generation(params["lam"], A_p2s1, A_p2s1, A_p2s2)
            prob_p2s2 = probabilities_generation(params["lam"], A_p2s2, A_p2s1, A_p2s2)
            # Select strategies based on the generate probabilities
            strat_p1 = strategy_selection(prob_p1s1)
            strat_p2 = strategy_selection(prob_p2s1)
            # calculate payoff
            payoff_p1_s1, payoff_p1_s2, payoff_p2_s1, payoff_p2_s2 = payoffs_generation(
                s1_array, s2_array, p1_mat, p2_mat, strat_p1, strat_p2
            )
            # Calculate weighted payoffs
            (
                p1_strategy,
                p2_strategy,
                wp_p1s1,
                wp_p1s2,
                wp_p2s1,
                wp_p2s2,
            ) = weighted_payoffs_generation(
                strat_p1,
                strat_p2,
                payoff_p1_s1,
                payoff_p1_s2,
                payoff_p2_s1,
                payoff_p2_s2,
                params["delta"],
            )
            # Check which payoff was realized and add to the cumulative
            if p1_strategy == 1:
                cum_payoff_p1 = cum_payoff_p1 + payoff_p1_s1
                payoff_p1 = payoff_p1_s1
            else:
                cum_payoff_p1 = cum_payoff_p1 + payoff_p1_s2
                payoff_p1 = payoff_p1_s2

            if p2_strategy == 1:
                cum_payoff_p2 = cum_payoff_p2 + payoff_p2_s1
                payoff_p2 = payoff_p2_s1
            else:
                cum_payoff_p2 = cum_payoff_p2 + payoff_p2_s2
                payoff_p2 = payoff_p2_s2

            # Attractions (A) and experience (N) update
            A_p1s1, A_p1s2, A_p2s1, A_p2s2, N = ewa_update(
                params["rho"],
                A_p1s1,
                A_p1s2,
                A_p2s1,
                A_p2s2,
                wp_p1s1,
                wp_p1s2,
                wp_p2s1,
                wp_p2s2,
                N,
                params["phi"],
            )
            
            
            # find intermediate loss
            intermediate_Q = np.sqrt(((prob_p1s1 - real_prob_p1s1) ** 2 + (prob_p2s1 - real_prob_p2s1) ** 2))

            round_nr += 1
            out_row = [
                sim_nr,
                round_nr,
                prob_p1s1,
                prob_p1s2,
                prob_p2s1,
                prob_p2s2,
                p1_strategy,
                p2_strategy,
                payoff_p1,
                payoff_p2,
                payoff_p1_s1,
                payoff_p1_s2,
                payoff_p2_s1,
                payoff_p2_s2,
                wp_p1s1,
                wp_p1s2,
                wp_p2s1,
                wp_p2s2,
                cum_payoff_p1,
                cum_payoff_p2,
                N,
                A_p1s1,
                A_p1s2,
                A_p2s1,
                A_p2s2,
                intermediate_Q,
            ]

            # adding the refined row into the dataframe
            out_all.append(out_row)
            
            if trial is not None:
                # report intermediate results to optuna
                trial.report(intermediate_Q, round_nr)
                
                if trial.should_prune():
                    raise optuna.TrialPruned()
            else:
                # This is a normal run and not a hyperparameter optimization one.
                # Early stop the trial if no improvement in loss
                current_loss = intermediate_Q
                # prev_loss = out_row[-2][-1]
                if current_loss > prev_loss:
                    trigger_times += 1
                    if trigger_times >= params["patience"]:
                        # Early stop the simulation
                        break
                else:
                    trigger_times = 0
                prev_loss = current_loss
        else:
            sim_nr += 1
            continue
        break
            
    simulation_df = pd.DataFrame(out_all)
    simulation_df.columns = [
        "sim_nr",
        "round_nr",
        "prob_p1s1",
        "prob_p1s2",
        "prob_p2s1",
        "prob_p2s2",
        "p1_strategy",
        "p2_strategy",
        "payoff_p1",
        "payoff_p2",
        "payoff_p1_s1",
        "payoff_p1_s2",
        "payoff_p2_s1",
        "payoff_p2_s2",
        "wp_p1s1",
        "wp_p1s2",
        "wp_p2s1",
        "wp_p2s2",
        "cum_payoff_p1",
        "cum_payoff_p2",
        "N",
        "A_p1s1",
        "A_p1s2",
        "A_p2s1",
        "A_p2s2",
        "Q",
    ]
    #simulation_df.to_csv("simulation_data.csv", index=False) #uncomment this line for exporing the csv file
    # print("Simulation finished, data file saved to the specified repository.")
    
    return simulation_df


def objective(trial: optuna.Trial):
    
    params = {
        "rho": trial.suggest_float("rho", 0, 1),
        "delta": trial.suggest_float("delta", 0, 1),
        "phi": trial.suggest_float("phi", 0, 1),
        "lam": trial.suggest_float("lam", 0, 1),
        "patience": 10,
    }
    
    simulation_output = ewa_simulation(n_sims, n_periods, A_0_p1s1, A_0_p1s2, A_0_p2s1, A_0_p2s2, N_0, params, trial=trial)
    # simulation_output.loc[simulation_output["Q"].argmin()]
    # loss = simulation_output["Q"].min()
    # Loss of last round
    loss = simulation_output["Q"].iloc[-1]
    return loss

study = optuna.create_study(
    direction='minimize', 
    pruner=optuna.pruners.PatientPruner(optuna.pruners.MedianPruner(), patience=10),
)
study.optimize(objective, n_trials=200, n_jobs=12)

# print('Data preview: ')
# simulation_output.head(10)

In [17]:
# Best trial
print("Best parameters:", study.best_trial.params)
print("Best loss:", study.best_trial.value)

Best parameters {'rho': 0.9094555419510801, 'delta': 0.9365419141643361, 'phi': 0.5115671387884919, 'lam': 2.480847979828213e-05}
Best loss 6.741385928217915e-07


In [19]:
def get_trial_results(trial):
    # define your trial result dictionary here
    return {'Trial Number': trial.number,
            'Value': trial.value,
            'Params': trial.params}
    
results = []
for trial in study.trials:
    results.append(get_trial_results(trial))

trials_df = pd.DataFrame(results)
trials_df
# df.to_csv('trial_results.csv', index=False)

,Trial Number,Value,Params
0,0,0.001001,"{'rho': 0.9931419489739164, 'delta': 0.9224319..."
1,1,0.202331,"{'rho': 0.5012834067992868, 'delta': 0.2921689..."
2,2,0.210787,"{'rho': 0.1483036449031806, 'delta': 0.3117715..."
3,3,0.185622,"{'rho': 0.12565896334067506, 'delta': 0.596346..."
4,4,0.045836,"{'rho': 0.439372530564605, 'delta': 0.66515162..."
...,...,...,...
165,165,0.000794,"{'rho': 0.9707511659739293, 'delta': 0.9099514..."
166,166,0.000117,"{'rho': 0.9714332526309604, 'delta': 0.9033003..."
167,167,0.000640,"{'rho': 0.972247192779372, 'delta': 0.77377800..."
168,168,0.001022,"{'rho': 0.9698223962156172, 'delta': 0.8261590..."


In [22]:
## Running using the best parameters
params = study.best_params
params["patience"] = 10

simulation_output = ewa_simulation(n_sims, n_periods, A_0_p1s1, A_0_p1s2, A_0_p2s1, A_0_p2s2, N_0, params)
loss = simulation_output["Q"].iloc[-1]
print("Loss:", loss)

Loss: 3.1704912925279825e-06


In [23]:
simulation_output

,sim_nr,round_nr,prob_p1s1,prob_p1s2,prob_p2s1,prob_p2s2,p1_strategy,p2_strategy,payoff_p1,payoff_p2,...,wp_p2s1,wp_p2s2,cum_payoff_p1,cum_payoff_p2,N,A_p1s1,A_p1s2,A_p2s1,A_p2s2,Q
0,1,1,0.500006,0.499994,0.499988,0.500012,2,1,0,0,...,0.000000,-1.873084,0,0,1.909456,0.758388,0.000000,-0.535825,-0.980952,0.000014
1,1,2,0.500005,0.499995,0.500003,0.499997,2,2,2,-2,...,0.000000,-2.000000,2,-2,2.736565,0.270706,0.730843,-0.191262,-1.080993,0.000005
2,1,3,0.499997,0.500003,0.500006,0.499994,2,2,2,-2,...,0.000000,-2.000000,4,-4,3.488784,0.108626,0.866529,-0.076747,-1.007033,0.000006
3,1,4,0.499995,0.500005,0.500006,0.499994,1,1,1,-1,...,-1.000000,0.000000,5,-5,4.172894,0.286101,0.370615,-0.272467,-0.430708,0.000007
4,1,5,0.499999,0.500001,0.500001,0.499999,2,2,2,-2,...,0.000000,-2.000000,7,-7,4.795062,0.127369,0.582090,-0.121300,-0.608843,0.000001
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34678,35,679,0.499998,0.500002,0.500001,0.499999,2,2,2,-2,...,0.000000,-2.000000,527,-527,11.044298,0.000969,0.363225,-0.042011,-0.281142,0.000002
34679,35,680,0.499998,0.500002,0.500001,0.499999,2,2,2,-2,...,0.000000,-2.000000,529,-529,11.044298,0.000496,0.366903,-0.021491,-0.324912,0.000003
34680,35,681,0.499998,0.500002,0.500002,0.499998,2,2,2,-2,...,0.000000,-2.000000,531,-531,11.044298,0.000254,0.368784,-0.010994,-0.347303,0.000003
34681,35,682,0.499998,0.500002,0.500002,0.499998,2,2,2,-2,...,0.000000,-2.000000,533,-533,11.044298,0.000130,0.369747,-0.005624,-0.358758,0.000003
